# Finding Numerical Fluxes for DG

Copyright (C) 2020 Andreas Kloeckner

<details>
<summary>MIT License</summary>
Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in
all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
THE SOFTWARE.
</details>

----
NB: This note book uses the [Maxima-Jupyter kernel](https://github.com/robert-dodier/maxima-jupyter) to interface with [Maxima](http://maxima.sourceforge.net/), an open-source computer algebra system. I have found that the [Docker image](https://hub.docker.com/r/calyau/maxima-jupyter) is a fairly convenient way of using this kernel.

Some Maxima version info for reproducibility:

In [106]:
build_info();

(%o53) 
Maxima version: "5.43.0"
Maxima build date: "2019-06-09 14:40:12"
Host type: "x86_64-unknown-linux-gnu"
Lisp implementation type: "SBCL"
Lisp implementation version: "1.5.3"
User dir: "/home/oubiwann/.maxima"
Temp dir: "/tmp"
Object dir: "/home/oubiwann/.maxima/binary/5_43_0/sbcl/1_5_3"
Frontend: false

Load some packages:

In [88]:
kill(all);
load("itensor");

assert(condition):=if not condition then error("Assertion violated") else true$

norm_2_squared(v):=v.v;

crossfunc(f):=makelist(
  sum(sum(
      levi_civita([i,j,k])*f(j,k),
   j,1,3),k,1,3),i,1,3)$

crossprod(a,b):=crossfunc(lambda([j,k], a[j]*b[k]));


REDEFINITION-WITH-DEFMACRO: 
  redefining MAXIMA::IFNOT in DEFMACRO

REDEFINITION-WITH-DEFMACRO: 
  redefining MAXIMA::M+OR*OR^P in DEFMACRO

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::$IDUMMY-IMPL in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::$IDUMMY in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::ISPROD in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::DERAT in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::PLUSI in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::MINUSI in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::COVI in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::CONTI in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::DERI in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::NAME in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::$COVI-IMPL in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::$COVI in DEFUN

REDEFINITION-WITH-DEFUN: 
  redefining MAXIMA::$CONTI-IMPL in DEFUN

REDEFINITION-WITH-D

(%o0)                                done

(%o1)         /usr/share/maxima/5.43.0/share/tensor/itensor.lisp

(%o3)                     norm_2_squared(v) := v . v

(%o5)         crossprod(a, b) := crossfunc(lambda([j, k], a  b ))
                                                           j  k

## Simplification Utilities

These function simplify expressions coming from the (symbolic) simultaneous diagonalization, by letting Maxima realize that $n \cdot n =1$.

In [89]:
/* ------------------------------------------------------------------------- */
/* Simplification for expressions stemming from hyperbolic systems */
/* ------------------------------------------------------------------------- */

hypsimp(x):=ratsimp(ratsubst(1,n.n,x))$

fullhypsimp(x):=hypsimp(
  ratsubst(
    last(n)^2,
    1-sum(n[i]^2,i,1,length(n)-1),
    x)
  )$
  
/* ------------------------------------------------------------------------- */
/* diagonalize a given hyperbolic operator A */
/* ------------------------------------------------------------------------- */

hypdiagonalize(A):=block([evA, V, invV,D],
  evA:hypsimp(apply(append, eigenvectors(A)[2])),
  V:transpose(apply(matrix, evA)),
  invV:hypsimp(invert(V)),
  assert(hypsimp(V.invV)=ident(length(A))),
  D:hypsimp(invV.A.V),
  [V, D, invV])$

## Wave Equation

In [90]:
/* redefine this to change dimensionality: */
n:[nx,ny]; 

(%o9)                              [nx, ny]

In [91]:
dims:length(n);

assume(c>0);

if dims = 1 then n:[1];


(%o10)                                 2

(%o11)                              [c > 0]

(%o12)                               false

Define the flux jacobian for the first-order form of the wave-equation:

$$
\begin{align*}
\partial_t u &= \nabla \cdot \boldsymbol v\\
\partial_t \boldsymbol v &= \nabla u
\end{align*}
$$
projected onto a line with normal `n`:

In [92]:
esymmatrix(n, v, i,j):=ematrix(n,n,v,i,j)+ematrix(n,n,v,j,i);
wave_A:sum(n[i]*esymmatrix(dims+1, -c, 1+i,1),i,1,dims);

(%o13) esymmatrix(n, v, i, j) := ematrix(n, n, v, i, j)
                                                       + ematrix(n, n, v, j, i)

                          [   0     - c nx  - c ny ]
                          [                        ]
(%o14)                    [ - c nx    0       0    ]
                          [                        ]
                          [ - c ny    0       0    ]

Find the eigenvalues of the flux Jacobian:

In [93]:
[wave_V, wave_D, wave_invV]:hypdiagonalize(wave_A);

                                             [ 1    nx       ny    ]
          [ 1    1     0   ]                 [ -    --       --    ]
          [                ]  [ - c  0  0 ]  [ 2    2        2     ]
          [ nx  - nx   1   ]  [           ]  [                     ]
(%o15)   [[                ], [  0   c  0 ], [ 1     nx       ny   ]]
          [             nx ]  [           ]  [ -   - --     - --   ]
          [ ny  - ny  - -- ]  [  0   0  0 ]  [ 2     2        2    ]
          [             ny ]                 [                     ]
                                             [          2          ]
                                             [ 0  1 - nx   - nx ny ]

### Finding the Numerical Flux

This function sets up a system of Rankine-Hugoniot conditions across the flux fan and solves for the stagnation state:

In [94]:
/* ------------------------------------------------------------------------- */
/* compute upwind flux for a given operator with eigenvalues evs, sorted
 * in ascending order.
 * Sign assumptions for all variables occuring in evs must be in place.
 */
/* ------------------------------------------------------------------------- */
hyp_upwind_flux(evs, D):=block([evvars, Dp, Dm, n, midstates, states, unknowns],
  evvars:listofvars(evs),

  add_evvars_suffix(suffix, x):=subst(makelist(v=concat(''v, suffix), v, evvars), x),

  evsm:add_evvars_suffix(m, evs),
  evsp:add_evvars_suffix(p, evs),

  Dm:add_evvars_suffix(m, D),
  Dp:add_evvars_suffix(p, D),

  midstates:makelist(makelist(concat(s,state,i), i, 1, length(D)),
      state, 1, length(evs)-1),

  states:append(
    [makelist(concat(sm, i), i, 1, length(D))],
    midstates,
    [makelist(concat(sp,i), i, 1, length(D))]),

  unknowns:apply(append, midstates),

  result:if member(0, evs) then
    block([biasedD, veceqns, eqns, soln],
      biasedD:makelist(
        if evs[i] = 0 then [Dp,Dm]
        else if evs[i] > 0 then [Dp,Dp]
        else [Dm,Dm],
        i, 1, length(evs)),

      veceqns:apply(append, makelist(
        -(if evs[i] > 0 then evsp[i] else evsm[i]) *(states[i+1]-states[i])
        +(biasedD[i][1].states[i+1]-biasedD[i][2].states[i]),
        i,1,length(evs))),

      eqns:makelist(veceqns[i,1], i, 1, length(veceqns)),

      soln:solve(eqns, unknowns),
      assert(length(soln)=1),

      for i: 1 thru length(evs) do
        if evs[i] = 0 then return(Dp.subst(soln[1], midstates[i]))
    )
  else
    block([straddle_idx, Dstates, veceqns, eqns, soln],
      straddle_idx:for i: 1 thru length(evs)-1 do
        if (evs[i] < 0) and (evs[i+1] > 0) then return(i),

      flux:makelist(concat(flux,i),i,1,length(D)),

      unknowns:append(unknowns, flux),

      Dstates:append(
        [Dm.first(states)],
        makelist(
          if i = straddle_idx then flux
          else if evs[i] > 0 then Dp.midstates[i]
          else Dm.midstates[i],
          i, 1, length(midstates)),
        [Dp.last(states)]),

      veceqns:apply(append, makelist(
        -(if evs[i] > 0 then evsp[i] else evsm[i]) *(states[i+1]-states[i])
        +(Dstates[i+1]-Dstates[i]),
        i,1,length(evs))),

      eqns:makelist(veceqns[i,1], i, 1, length(veceqns)),

      print(covect(eqns)),
      soln:solve(eqns, unknowns),
      assert(length(soln)=1),

      subst(soln[1], flux)
    ),
  subst(
    append(
      makelist(concat(sm, i)=sm[i,1], i, 1, length(D)),
      makelist(concat(sp, i)=sp[i,1], i, 1, length(D))
      ),
    result)
  )$

Find an expression for the flux in characteristic variables.

Note the `p` and `m` suffixes for the $+$ and $-$ sides of the interface.

In [95]:
wave_eigenvalues:makelist(wave_D[i,i], i, 1, length(wave_D));

if member(0, wave_eigenvalues) then
  wave_sflux:hyp_upwind_flux([-c,0,c], wave_D)
else
  wave_sflux:hyp_upwind_flux([-c,c], wave_D);



solve: dependent equations eliminated: (1 6 8)


(%o17)                            [- c, c, 0]

                                [ - sp     cp ]
                                [     1, 1    ]
                                [             ]
(%o18)                          [  sm     cm  ]
                                [    2, 1     ]
                                [             ]
                                [      0      ]

Convert back to conserved variables:

In [96]:
wave_wflux:ratsimp(wave_V.ev(wave_sflux, [sm=wave_sminw,sp=wave_spinw]));

              [     wave_sminw     cm - wave_spinw     cp      ]
              [               2, 1                1, 1         ]
              [                                                ]
(%o19)        [ ((- wave_spinw     cp) - wave_sminw     cm) nx ]
              [               1, 1                 2, 1        ]
              [                                                ]
              [ ((- wave_spinw     cp) - wave_sminw     cm) ny ]
              [               1, 1                 2, 1        ]

## Maxwell's Equations

First, set up some parameter assumptions:

In [97]:
assume(c>0);
assume(mu>0);
assume(epsilon>0);
assume(epsinv>0);
assume(muinv>0);

(%o20)                            [redundant]

(%o21)                             [mu > 0]

(%o22)                           [epsilon > 0]

(%o23)                           [epsinv > 0]

(%o24)                            [muinv > 0]

Some helper functions for matrix creation:

In [98]:
/* A hyperbolic system matrix resulting from a curl */
curlmat(coord):=genmatrix(
  lambda ([i,j], levi_civita([coord,j,i])),
  3,3)$

vstack:append$

hstack(a,b):=transpose(append(transpose(a),transpose(b)))$

blockmat(a11,a12,a21,a22):=vstack(hstack(a11,a12),hstack(a21,a22))$

In [99]:
n:[nx,ny,nz];

(%o29)                           [nx, ny, nz]

Next, write down the flux Jacobian on a line with normal `n`:

In [100]:
max_submat(i):=blockmat(
  zeromatrix(3,3),
  -epsinv*curlmat(i), /* epsinv = 1/epsilon */
  muinv*curlmat(i), /* muinv = 1/mu */
  zeromatrix(3,3)
  )$

max_Asimp:sum(n[i]*max_submat(i),i,1,3);
max_A:subst([epsinv=1/epsilon,muinv=1/mu], max_Asimp);

(%o31) 
  [     0           0           0            0        epsinv nz   - epsinv ny ]
  [                                                                           ]
  [     0           0           0       - epsinv nz       0        epsinv nx  ]
  [                                                                           ]
  [     0           0           0        epsinv ny   - epsinv nx       0      ]
  [                                                                           ]
  [     0       - muinv nz   muinv ny        0            0            0      ]
  [                                                                           ]
  [  muinv nz       0       - muinv nx       0            0            0      ]
  [                                                                           ]
  [ - muinv ny   muinv nx       0            0            0            0      ]

             [                                 nz          ny    ]
             [  0     0     0        0       -------   - ------- ]
             [                               epsilon     epsilon ]
             [                                                   ]
             [                       nz                   nx     ]
             [  0     0     0    - -------      0       -------  ]
             [                     epsilon              epsilon  ]
             [                                                   ]
             [                      ny          nx               ]
             [  0     0     0     -------   - -------      0     ]
             [                    epsilon     epsilon            ]
(%o32)       [                                                   ]
             [         nz   ny                                   ]
             [  0    - --   --       0          0          0     ]
             [         mu   mu                                

Next, diagonalize to obtain the transformation to/from characteristic variables:

In [101]:
max_invsubst(x):=subst([epsinv=1/epsilon, muinv=1/mu], x)$

[max_V, max_D, max_invV]:max_invsubst(hypdiagonalize(max_Asimp));

                                                ny
(%o34) [matrix([1, 0, 1, 0, 1, 0], [0, 1, 0, 1, --, 0], 
                                                nx
   nx    ny    nx    ny  nz       sqrt(epsilon) nx ny
[- --, - --, - --, - --, --, 0], [-------------------, 
   nz    nz    nz    nz  nx           sqrt(mu) nz
                   2                                               2
  sqrt(epsilon) (nx  - 1)    sqrt(epsilon) nx ny  sqrt(epsilon) (nx  - 1)
- -----------------------, - -------------------, -----------------------, 0, 
        sqrt(mu) nz              sqrt(mu) nz            sqrt(mu) nz
                      2
     sqrt(epsilon) (ny  - 1)    sqrt(epsilon) nx ny
1], [-----------------------, - -------------------, 
           sqrt(mu) nz              sqrt(mu) nz
                   2
  sqrt(epsilon) (ny  - 1)  sqrt(epsilon) nx ny     ny
- -----------------------, -------------------, 0, --], 
        sqrt(mu) nz            sqrt(mu) nz         nx
 sqrt(epsilon) ny    sqrt(e

Now find the flux in characteristic variables:

In [102]:
max_Dinc:subst([1/(sqrt(epsilon)*sqrt(mu))=c], max_D);
max_sflux:hyp_upwind_flux([-c,0,c], max_Dinc);

/* FIXME: max_V should not depend on epsilon and mu, but it does 
   For now, make cp and cm equal. */

max_sflux:subst(
  [cp=1/(sqrt(epsilon)*sqrt(mu)), cm=1/(sqrt(epsilon)*sqrt(mu))], 
  max_sflux);


solve: dependent equations eliminated: (1 2 11 16 15 12)


                           [ - c   0   0  0  0  0 ]
                           [                      ]
                           [  0   - c  0  0  0  0 ]
                           [                      ]
                           [  0    0   c  0  0  0 ]
(%o35)                     [                      ]
                           [  0    0   0  c  0  0 ]
                           [                      ]
                           [  0    0   0  0  0  0 ]
                           [                      ]
                           [  0    0   0  0  0  0 ]

                                [ - sp     cp ]
                                [     1, 1    ]
                                [             ]
                                [ - sp     cp ]
                                [     2, 1    ]
                                [             ]
                                [  sm     cm  ]
(%o36)                          [    3, 1     ]
                                [             ]
                                [  sm     cm  ]
                                [    4, 1     ]
                                [             ]
                                [      0      ]
                                [             ]
                                [      0      ]

                         [           sp             ]
                         [             1, 1         ]
                         [ - ---------------------- ]
                         [   sqrt(epsilon) sqrt(mu) ]
                         [                          ]
                         [           sp             ]
                         [             2, 1         ]
                         [ - ---------------------- ]
                         [   sqrt(epsilon) sqrt(mu) ]
                         [                          ]
                         [          sm              ]
(%o37)                   [            3, 1          ]
                         [  ----------------------  ]
                         [  sqrt(epsilon) sqrt(mu)  ]
                         [                          ]
                         [          sm              ]
                         [            4, 1          ]
                         [  ----------------------  ]
                         [  

And in conserved variables:

In [103]:
max_Em:makelist(Em[i],i,1,3)$
max_Ep:makelist(Ep[i],i,1,3)$
max_Hm:makelist(Hm[i],i,1,3)$
max_Hp:makelist(Hp[i],i,1,3)$
max_wm:vstack(max_Em,max_Hm)$
max_wp:vstack(max_Ep,max_Hp)$

max_sminw:hypsimp(max_invV.max_wm)$
max_spinw:hypsimp(max_invV.max_wp)$

max_wflux:fullhypsimp(max_V.ev(max_sflux, [sm=max_sminw,sp=max_spinw]));

(%o46) matrix([(sqrt(epsilon) ((Ep  - Em ) nx nz + (Ep  - Em ) nx ny
                                  3     3             2     2
                 2
 + (Ep  - Em ) nx  - Ep  + Em ) + sqrt(mu)
      1     1          1     1
 ((Hp  + Hm ) nz + ((- Hp ) - Hm ) ny))/(2 epsilon sqrt(mu))], 
     2     2             3      3
                                                     2
[- (sqrt(epsilon) ((Em  - Ep ) ny nz + (Em  - Ep ) ny  + (Em  - Ep ) nx ny
                      3     3             2     2           1     1
 + Ep  - Em ) + sqrt(mu) ((Hp  + Hm ) nz + ((- Hp ) - Hm ) nx))
     2     2                 1     1             3      3
/(2 epsilon sqrt(mu))], [(sqrt(epsilon)
                                                       2                 2
 (((Ep  - Em ) ny + (Ep  - Em ) nx) nz + (Em  - Ep ) ny  + (Em  - Ep ) nx )
      2     2          1     1              3     3           3     3
 + sqrt(mu) ((Hp  + Hm ) ny + ((- Hp ) - Hm ) nx))/(2 epsilon sqrt(mu))], 
                1     

In [104]:
max_stronglocalpart:max_A.max_wm;

max_strongwflux:max_stronglocalpart-max_wflux;

                             [ Hm  nz    Hm  ny  ]
                             [   2         3     ]
                             [ ------- - ------- ]
                             [ epsilon   epsilon ]
                             [                   ]
                             [ Hm  nx    Hm  nz  ]
                             [   3         1     ]
                             [ ------- - ------- ]
                             [ epsilon   epsilon ]
                             [                   ]
                             [ Hm  ny    Hm  nx  ]
                             [   1         2     ]
                             [ ------- - ------- ]
                             [ epsilon   epsilon ]
(%o47)                       [                   ]
                             [  Em  ny   Em  nz  ]
                             [    3        2     ]
                             [  ------ - ------  ]
                             [    mu       mu    ]
                             [ 

(%o48) matrix([(- (sqrt(epsilon) ((Ep  - Em ) nx nz + (Ep  - Em ) nx ny
                                     3     3             2     2
                 2
 + (Ep  - Em ) nx  - Ep  + Em ) + sqrt(mu)
      1     1          1     1
                                                                Hm  nz
                                                                  2
 ((Hp  + Hm ) nz + ((- Hp ) - Hm ) ny))/(2 epsilon sqrt(mu))) + -------
     2     2             3      3                               epsilon
   Hm  ny
     3                                                          2
 - -------], [(sqrt(epsilon) ((Em  - Ep ) ny nz + (Em  - Ep ) ny
   epsilon                       3     3             2     2
 + (Em  - Ep ) nx ny + Ep  - Em ) + sqrt(mu)
      1     1            2     2
                                                               Hm  nz
                                                                 1
 ((Hp  + Hm ) nz + ((- Hp ) - Hm ) nx))/(2 epsilon sqrt(mu)) - -------

Check against value from [the literature](https://doi.org/10.1016/0010-4655(91)90199-U):

In [105]:
max_Z:sqrt(mu/epsilon)$
max_Y:sqrt(epsilon/mu)$

max_knownstrongwflux:ratsimp(vstack(
  -1/(2*epsilon)
  *(crossprod(n,(max_Hm-max_Hp)-1/max_Z*crossprod(n,max_Em-max_Ep))),
  1/(2*mu)
  *(crossprod(n,(max_Em-max_Ep)+1/max_Y*crossprod(n,max_Hm-max_Hp)))
  ));

assert(norm_2_squared(hypsimp(
  (max_strongwflux)
  -max_knownstrongwflux))=0);

                                               2
(%o51) [(sqrt(epsilon) sqrt(mu) ((Ep  - Em ) nz  + (Em  - Ep ) nx nz
                                    1     1           3     3
                 2
 + (Ep  - Em ) ny  + (Em  - Ep ) nx ny) + (Hm  - Hp ) mu nz
      1     1           2     2              2     2
 + (Hp  - Hm ) mu ny)/(2 epsilon mu), (sqrt(epsilon) sqrt(mu)
      3     3
                2                                                         2
 ((Ep  - Em ) nz  + (Em  - Ep ) ny nz + (Em  - Ep ) nx ny + (Ep  - Em ) nx )
     2     2           3     3             1     1             2     2
 + (Hp  - Hm ) mu nz + (Hm  - Hp ) mu nx)/(2 epsilon mu), 
      1     1             3     3
- (sqrt(epsilon) sqrt(mu) (((Ep  - Em ) ny + (Ep  - Em ) nx) nz
                               2     2          1     1
                 2                 2
 + (Em  - Ep ) ny  + (Em  - Ep ) nx ) + (Hp  - Hm ) mu ny + (Hm  - Hp ) mu nx)
      3     3           3     3            1     1         

(%o52)                               true